**Exploring Tweet Pattern based on type of tweet**


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp "/content/drive/MyDrive/Cresci17/cresci-2017.csv.zip" "data.zip"

!unzip data.zip
!unzip /content/datasets_full.csv/crowdflower_results.csv.zip
!unzip /content/datasets_full.csv/fake_followers.csv.zip
!unzip /content/datasets_full.csv/genuine_accounts.csv.zip
!unzip /content/datasets_full.csv/social_spambots_1.csv.zip
!unzip /content/datasets_full.csv/social_spambots_2.csv.zip
!unzip /content/datasets_full.csv/social_spambots_3.csv.zip
!unzip /content/datasets_full.csv/social_spambots_4.csv.zip
!unzip /content/datasets_full.csv/traditional_spambots_1.csv.zip
!unzip /content/datasets_full.csv/traditional_spambots_2.csv.zip
!unzip /content/datasets_full.csv/traditional_spambots_3.csv.zip
!unzip /content/datasets_full.csv/traditional_spambots_4.csv.zip

Archive:  data.zip
   creating: datasets_full.csv/
  inflating: datasets_full.csv/crowdflower_results.csv.zip  
  inflating: datasets_full.csv/fake_followers.csv.zip  
  inflating: datasets_full.csv/genuine_accounts.csv.zip  
  inflating: datasets_full.csv/social_spambots_1.csv.zip  
  inflating: datasets_full.csv/social_spambots_2.csv.zip  
  inflating: datasets_full.csv/social_spambots_3.csv.zip  
  inflating: datasets_full.csv/traditional_spambots_1.csv.zip  
  inflating: datasets_full.csv/traditional_spambots_2.csv.zip  
  inflating: datasets_full.csv/traditional_spambots_3.csv.zip  
  inflating: datasets_full.csv/traditional_spambots_4.csv.zip  
  inflating: READ.ME                 
  inflating: datasets_full.csv/READ.ME  
Archive:  /content/datasets_full.csv/crowdflower_results.csv.zip
   creating: crowdflower_results.csv/
  inflating: crowdflower_results.csv/crowdflower_results_aggregated.csv  
   creating: __MACOSX/
   creating: __MACOSX/crowdflower_results.csv/
  inflating: __

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns

In [5]:
import random
import sys
import time
import zlib

In [6]:
from sklearn import utils
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, confusion_matrix
from scipy.stats import ttest_1samp

In [7]:
### Users

# Genuine accounts.
gen = pd.read_csv('/content/genuine_accounts.csv/users.csv')

# Social spambots.
ss1 = pd.read_csv('/content/social_spambots_1.csv/users.csv')
ss2 = pd.read_csv('/content/social_spambots_2.csv/users.csv')
ss3 = pd.read_csv('/content/social_spambots_3.csv/users.csv')

# Traditional spambots.
ts1 = pd.read_csv('/content/traditional_spambots_1.csv/users.csv')
ts2 = pd.read_csv('/content/traditional_spambots_2.csv/users.csv')
ts3 = pd.read_csv('/content/traditional_spambots_3.csv/users.csv')
ts4 = pd.read_csv('/content/traditional_spambots_4.csv/users.csv')

In [13]:
gen

,id,name,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,url,lang,...,notifications,description,contributors_enabled,following,created_at,timestamp,crawled_at,updated,test_set_1,test_set_2
0,1502026416,TASUKU HAYAKAWA,0918Bask,2177,208,332,265,1,NaN,ja,...,NaN,15years ago X.Lines24,NaN,NaN,Tue Jun 11 11:20:35 +0000 2013,2013-06-11 13:20:35,2015-05-02 06:41:46,2016-03-15 15:53:47,0,0
1,2492782375,ro_or,1120Roll,2660,330,485,3972,5,NaN,ja,...,NaN,保守見習い地元大好き人間。 経済学、電工、仏教を勉強中、ちなDeではいかんのか？ (*^◯^*),NaN,NaN,Tue May 13 10:37:57 +0000 2014,2014-05-13 12:37:57,2015-05-01 17:20:27,2016-03-15 15:53:48,0,0
2,293212315,bearclaw,14KBBrown,1254,166,177,1185,0,NaN,en,...,NaN,Let me see what your best move is!,NaN,NaN,Wed May 04 23:30:37 +0000 2011,2011-05-05 01:30:37,2015-05-01 18:48:28,2016-03-15 15:53:48,0,0
3,191839658,pocahontas farida,wadespeters,202968,2248,981,60304,101,http://t.co/rGV0HIJGsu,en,...,NaN,20. menna: #farida #nyc and the 80s actually y...,NaN,NaN,Fri Sep 17 14:02:10 +0000 2010,2010-09-17 16:02:10,2015-05-01 13:55:16,2016-03-15 15:53:48,0,0
4,3020965143,Ms Kathy,191a5bd05da04dc,82,21,79,5,0,NaN,en,...,NaN,Cosmetologist,NaN,NaN,Fri Feb 06 04:10:49 +0000 2015,2015-02-06 05:10:49,2015-05-02 01:17:32,2016-03-15 15:53:48,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3469,205218909,Alejandro,zombiemaster999,315,94,597,36,4,NaN,es,...,NaN,NaN,NaN,NaN,Wed Oct 20 11:42:54 +0000 2010,2010-10-20 13:42:54,2015-05-01 14:23:21,2016-03-15 16:11:20,0,0
3470,2874966164,Zubair Niazi Ⓜ️,zubairniaziPTI,4099,5378,1238,471,6,http://t.co/yuROtYHG8K,en,...,NaN,[ Member PTI SMT Official ] ( PTI District Pre...,NaN,NaN,Fri Oct 24 10:30:53 +0000 2014,2014-10-24 12:30:53,2015-05-01 22:20:13,2016-03-15 16:11:21,0,0
3471,2980901837,Zuhazuu,zuhazuu1,199,18,136,6,0,NaN,en,...,NaN,NaN,NaN,NaN,Fri Jan 16 09:59:43 +0000 2015,2015-01-16 10:59:43,2015-05-02 00:32:44,2016-03-15 16:11:21,0,0
3472,121122678,zveljka,zveljka,2609,41,263,121,0,NaN,en,...,NaN,Whatever it is you're seeking won't come in th...,NaN,NaN,Mon Mar 08 14:00:26 +0000 2010,2010-03-08 15:00:26,2015-05-02 08:15:25,2016-03-15 16:11:21,0,0


In [17]:
gen.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3474 entries, 0 to 3473
Data columns (total 42 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   id                                  3474 non-null   int64  
 1   name                                3473 non-null   object 
 2   screen_name                         3474 non-null   object 
 3   statuses_count                      3474 non-null   int64  
 4   followers_count                     3474 non-null   int64  
 5   friends_count                       3474 non-null   int64  
 6   favourites_count                    3474 non-null   int64  
 7   listed_count                        3474 non-null   int64  
 8   url                                 1266 non-null   object 
 9   lang                                3474 non-null   object 
 10  time_zone                           2475 non-null   object 
 11  location                            2365 no

In [20]:
gen.columns

Index(['id', 'name', 'screen_name', 'statuses_count', 'followers_count',
       'friends_count', 'favourites_count', 'listed_count', 'url', 'lang',
       'time_zone', 'location', 'default_profile', 'default_profile_image',
       'geo_enabled', 'profile_image_url', 'profile_banner_url',
       'profile_use_background_image', 'profile_background_image_url_https',
       'profile_text_color', 'profile_image_url_https',
       'profile_sidebar_border_color', 'profile_background_tile',
       'profile_sidebar_fill_color', 'profile_background_image_url',
       'profile_background_color', 'profile_link_color', 'utc_offset',
       'is_translator', 'follow_request_sent', 'protected', 'verified',
       'notifications', 'description', 'contributors_enabled', 'following',
       'created_at', 'timestamp', 'crawled_at', 'updated', 'test_set_1',
       'test_set_2'],
      dtype='object')

In [14]:
print(ss1.shape)
print(ss2.shape)
print(ss3.shape)
print(ts1.shape)

(991, 41)
(3457, 40)
(464, 41)
(1000, 40)


In [8]:
### Tweets

# Genuine accounts.
gen_tweets = pd.read_csv('/content/genuine_accounts.csv/tweets.csv', sep=",", encoding='Latin-1')

# Social spambots.
ss1_tweets = pd.read_csv('/content/social_spambots_1.csv/tweets.csv', sep=",", encoding='Latin-1')
ss2_tweets = pd.read_csv('/content/social_spambots_2.csv/tweets.csv', sep=",", encoding='Latin-1')
ss3_tweets = pd.read_csv('/content/social_spambots_3.csv/tweets.csv', sep=",", encoding='Latin-1')

# Traditional spambots.
ts1_tweets = pd.read_csv('/content/traditional_spambots_1.csv/tweets.csv', sep=",", encoding='Latin-1')

In [18]:
gen_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2839362 entries, 0 to 2839361
Data columns (total 25 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   id                       object 
 1   text                     object 
 2   source                   object 
 3   user_id                  float64
 4   truncated                float64
 5   in_reply_to_status_id    float64
 6   in_reply_to_user_id      float64
 7   in_reply_to_screen_name  object 
 8   retweeted_status_id      float64
 9   geo                      float64
 10  place                    object 
 11  contributors             float64
 12  retweet_count            float64
 13  reply_count              float64
 14  favorite_count           float64
 15  favorited                float64
 16  retweeted                float64
 17  possibly_sensitive       float64
 18  num_hashtags             float64
 19  num_urls                 float64
 20  num_mentions             float64
 21  created_

In [ ]:
gen_tweets

,id,text,source,user_id,truncated,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,retweeted_status_id,geo,...,favorited,retweeted,possibly_sensitive,num_hashtags,num_urls,num_mentions,created_at,timestamp,crawled_at,updated
0,593932392663912449,RT @morningJewshow: Speaking about Jews and co...,"<a href=""http://tapbots.com/tweetbot"" rel=""nof...",6.780330e+05,NaN,0.0,0.0,NaN,5.939322e+17,NaN,...,NaN,NaN,NaN,0.0,0.0,1.0,Fri May 01 00:18:11 +0000 2015,2015-05-01 02:18:11,2015-05-01 12:57:19,2015-05-01 12:57:19
1,593895316719423488,This age/face recognition thing..no reason pla...,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",6.780330e+05,NaN,0.0,0.0,NaN,0.000000e+00,NaN,...,NaN,NaN,NaN,0.0,0.0,0.0,Thu Apr 30 21:50:52 +0000 2015,2015-04-30 23:50:52,2015-05-01 12:57:19,2015-05-01 12:57:19
2,593880638069018624,Only upside of the moment I can think of is th...,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",6.780330e+05,NaN,0.0,0.0,NaN,0.000000e+00,NaN,...,NaN,NaN,NaN,2.0,0.0,0.0,Thu Apr 30 20:52:32 +0000 2015,2015-04-30 22:52:32,2015-05-01 12:57:19,2015-05-01 12:57:19
3,593847955536252928,If you're going to think about+create experien...,"<a href=""http://tapbots.com/tweetbot"" rel=""nof...",6.780330e+05,NaN,0.0,0.0,NaN,0.000000e+00,NaN,...,NaN,NaN,NaN,2.0,0.0,0.0,Thu Apr 30 18:42:40 +0000 2015,2015-04-30 20:42:40,2015-05-01 12:57:19,2015-05-01 12:57:19
4,593847687847350272,Watching a thread on FB about possible future ...,"<a href=""http://tapbots.com/tweetbot"" rel=""nof...",6.780330e+05,NaN,0.0,0.0,NaN,0.000000e+00,NaN,...,NaN,NaN,NaN,0.0,0.0,0.0,Thu Apr 30 18:41:36 +0000 2015,2015-04-30 20:41:36,2015-05-01 12:57:19,2015-05-01 12:57:19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2839357,546379158251929600,"""Hereâs the key to living a good life: âMe...","<a href=""http://www.tumblr.com/"" rel=""nofollow...",2.776254e+07,NaN,0.0,0.0,NaN,0.000000e+00,NaN,...,NaN,NaN,NaN,0.0,1.0,0.0,Sat Dec 20 18:58:37 +0000 2014,2014-12-20 19:58:37,2015-05-01 16:15:29,2015-05-01 16:15:29
2839358,578751592213700608,RT @Cuauhtemoc_1521: #SabiasQue el nombre real...,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",2.356426e+09,NaN,0.0,0.0,NaN,5.786470e+17,NaN,...,NaN,NaN,NaN,1.0,0.0,1.0,Fri Mar 20 02:55:06 +0000 2015,2015-03-20 03:55:06,2015-05-01 16:15:29,2015-05-01 16:15:29
2839359,546379003465314306,"""If someone says they have their shit together...","<a href=""http://www.tumblr.com/"" rel=""nofollow...",2.776254e+07,NaN,0.0,0.0,NaN,0.000000e+00,NaN,...,NaN,NaN,NaN,0.0,1.0,0.0,Sat Dec 20 18:58:00 +0000 2014,2014-12-20 19:58:00,2015-05-01 16:15:29,2015-05-01 16:15:29
2839360,588093827501527040,How to Train Your Coral: Imagine you are in th...,"<a href=""http://twitterfeed.com"" rel=""nofollow...",2.412403e+08,NaN,0.0,0.0,NaN,0.000000e+00,NaN,...,NaN,NaN,NaN,2.0,1.0,0.0,Tue Apr 14 21:37:49 +0000 2015,2015-04-14 23:37:49,2015-05-01 16:15:29,2015-05-01 16:15:29


In [19]:
ss1_tweets

,id,text,source,user_id,truncated,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,retweeted_status_id,geo,...,favorited,retweeted,possibly_sensitive,num_hashtags,num_urls,num_mentions,created_at,timestamp,crawled_at,updated
0,532627591686275072,I Pooh - In silenzio 1968 http://t.co/ahvQxUqTws,"<a href=""http://www.facebook.com/twitter"" rel=...",24858289,NaN,0,0,NaN,0,NaN,...,NaN,NaN,NaN,0,1,0,Wed Nov 12 20:14:48 +0000 2014,2014-11-12 21:14:48,2014-11-12 21:44:09,2014-11-12 21:44:09
1,532624255058706432,http://t.co/HyI5EQKz6Q,"<a href=""http://www.facebook.com/twitter"" rel=...",24858289,NaN,0,0,NaN,0,NaN,...,NaN,NaN,NaN,0,1,0,Wed Nov 12 20:01:32 +0000 2014,2014-11-12 21:01:32,2014-11-12 21:44:09,2014-11-12 21:44:09
2,532513524460052480,"Tutti a tavola, con il filetto di baccalÃ . ht...","<a href=""http://www.facebook.com/twitter"" rel=...",24858289,NaN,0,0,NaN,0,NaN,...,NaN,NaN,NaN,0,1,0,Wed Nov 12 12:41:32 +0000 2014,2014-11-12 13:41:32,2014-11-12 21:44:09,2014-11-12 21:44:09
3,532297646669852672,http://t.co/NAHQ4l2pUy,"<a href=""http://www.facebook.com/twitter"" rel=...",24858289,NaN,0,0,NaN,0,NaN,...,NaN,NaN,NaN,0,1,0,Tue Nov 11 22:23:43 +0000 2014,2014-11-11 23:23:43,2014-11-12 21:44:09,2014-11-12 21:44:09
4,532295960807100416,Gold - Spandau Ballet http://t.co/o8ZJHt7Neu,"<a href=""http://www.facebook.com/twitter"" rel=...",24858289,NaN,0,0,NaN,0,NaN,...,NaN,NaN,NaN,0,1,0,Tue Nov 11 22:17:01 +0000 2014,2014-11-11 23:17:01,2014-11-12 21:44:09,2014-11-12 21:44:09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1610029,470953862489468928,"""La fantasia Ã¨ la figlia diletta della libert...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",2525273432,NaN,0,0,NaN,0,NaN,...,NaN,NaN,NaN,0,0,0,Mon May 26 15:45:25 +0000 2014,2014-05-26 17:45:25,2014-11-14 01:40:34,2014-11-14 01:40:34
1610030,470953620155158528,RT @Piovegovernolad: INGHILTERRA: FARAGE LI HA...,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",2525273432,NaN,0,0,NaN,470924405070393344,NaN,...,NaN,NaN,NaN,0,1,1,Mon May 26 15:44:27 +0000 2014,2014-05-26 17:44:27,2014-11-14 01:40:34,2014-11-14 01:40:34
1610031,470953572868562945,"RT @Piovegovernolad: Kyenge eletta: ""Rappresen...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",2525273432,NaN,0,0,NaN,470893063049859074,NaN,...,NaN,NaN,NaN,0,1,1,Mon May 26 15:44:16 +0000 2014,2014-05-26 17:44:16,2014-11-14 01:40:34,2014-11-14 01:40:34
1610032,470953492358901760,RT @Piovegovernolad: Giulio Tremonti: âSe lâ...,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",2525273432,NaN,0,0,NaN,470644037784182784,NaN,...,NaN,NaN,NaN,0,1,1,Mon May 26 15:43:57 +0000 2014,2014-05-26 17:43:57,2014-11-14 01:40:34,2014-11-14 01:40:34


In [9]:
gen.shape

(3474, 42)

In [15]:
gen

,id,name,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,url,lang,...,notifications,description,contributors_enabled,following,created_at,timestamp,crawled_at,updated,test_set_1,test_set_2
0,1502026416,TASUKU HAYAKAWA,0918Bask,2177,208,332,265,1,NaN,ja,...,NaN,15years ago X.Lines24,NaN,NaN,Tue Jun 11 11:20:35 +0000 2013,2013-06-11 13:20:35,2015-05-02 06:41:46,2016-03-15 15:53:47,0,0
1,2492782375,ro_or,1120Roll,2660,330,485,3972,5,NaN,ja,...,NaN,保守見習い地元大好き人間。 経済学、電工、仏教を勉強中、ちなDeではいかんのか？ (*^◯^*),NaN,NaN,Tue May 13 10:37:57 +0000 2014,2014-05-13 12:37:57,2015-05-01 17:20:27,2016-03-15 15:53:48,0,0
2,293212315,bearclaw,14KBBrown,1254,166,177,1185,0,NaN,en,...,NaN,Let me see what your best move is!,NaN,NaN,Wed May 04 23:30:37 +0000 2011,2011-05-05 01:30:37,2015-05-01 18:48:28,2016-03-15 15:53:48,0,0
3,191839658,pocahontas farida,wadespeters,202968,2248,981,60304,101,http://t.co/rGV0HIJGsu,en,...,NaN,20. menna: #farida #nyc and the 80s actually y...,NaN,NaN,Fri Sep 17 14:02:10 +0000 2010,2010-09-17 16:02:10,2015-05-01 13:55:16,2016-03-15 15:53:48,0,0
4,3020965143,Ms Kathy,191a5bd05da04dc,82,21,79,5,0,NaN,en,...,NaN,Cosmetologist,NaN,NaN,Fri Feb 06 04:10:49 +0000 2015,2015-02-06 05:10:49,2015-05-02 01:17:32,2016-03-15 15:53:48,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3469,205218909,Alejandro,zombiemaster999,315,94,597,36,4,NaN,es,...,NaN,NaN,NaN,NaN,Wed Oct 20 11:42:54 +0000 2010,2010-10-20 13:42:54,2015-05-01 14:23:21,2016-03-15 16:11:20,0,0
3470,2874966164,Zubair Niazi Ⓜ️,zubairniaziPTI,4099,5378,1238,471,6,http://t.co/yuROtYHG8K,en,...,NaN,[ Member PTI SMT Official ] ( PTI District Pre...,NaN,NaN,Fri Oct 24 10:30:53 +0000 2014,2014-10-24 12:30:53,2015-05-01 22:20:13,2016-03-15 16:11:21,0,0
3471,2980901837,Zuhazuu,zuhazuu1,199,18,136,6,0,NaN,en,...,NaN,NaN,NaN,NaN,Fri Jan 16 09:59:43 +0000 2015,2015-01-16 10:59:43,2015-05-02 00:32:44,2016-03-15 16:11:21,0,0
3472,121122678,zveljka,zveljka,2609,41,263,121,0,NaN,en,...,NaN,Whatever it is you're seeking won't come in th...,NaN,NaN,Mon Mar 08 14:00:26 +0000 2010,2010-03-08 15:00:26,2015-05-02 08:15:25,2016-03-15 16:11:21,0,0


In [16]:
gen['id'].value_counts()

1502026416    1
1534422504    1
376585827     1
24386710      1
385226809     1
             ..
715457424     1
2364038797    1
3073742769    1
535206680     1
2910276853    1
Name: id, Length: 3474, dtype: int64

In [10]:
gen_tweets.shape

(2839362, 25)

In [ ]:
gen_tweets.dropna(axis = 0, inplace = True, subset = ['text', 'retweeted_status_id', 'in_reply_to_status_id'])

In [ ]:
gen_tweets

,id,text,source,user_id,truncated,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,retweeted_status_id,geo,...,favorited,retweeted,possibly_sensitive,num_hashtags,num_urls,num_mentions,created_at,timestamp,crawled_at,updated
0,593932392663912449,RT @morningJewshow: Speaking about Jews and co...,"<a href=""http://tapbots.com/tweetbot"" rel=""nof...",6.780330e+05,NaN,0.0,0.0,NaN,5.939322e+17,NaN,...,NaN,NaN,NaN,0.0,0.0,1.0,Fri May 01 00:18:11 +0000 2015,2015-05-01 02:18:11,2015-05-01 12:57:19,2015-05-01 12:57:19
1,593895316719423488,This age/face recognition thing..no reason pla...,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",6.780330e+05,NaN,0.0,0.0,NaN,0.000000e+00,NaN,...,NaN,NaN,NaN,0.0,0.0,0.0,Thu Apr 30 21:50:52 +0000 2015,2015-04-30 23:50:52,2015-05-01 12:57:19,2015-05-01 12:57:19
2,593880638069018624,Only upside of the moment I can think of is th...,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",6.780330e+05,NaN,0.0,0.0,NaN,0.000000e+00,NaN,...,NaN,NaN,NaN,2.0,0.0,0.0,Thu Apr 30 20:52:32 +0000 2015,2015-04-30 22:52:32,2015-05-01 12:57:19,2015-05-01 12:57:19
3,593847955536252928,If you're going to think about+create experien...,"<a href=""http://tapbots.com/tweetbot"" rel=""nof...",6.780330e+05,NaN,0.0,0.0,NaN,0.000000e+00,NaN,...,NaN,NaN,NaN,2.0,0.0,0.0,Thu Apr 30 18:42:40 +0000 2015,2015-04-30 20:42:40,2015-05-01 12:57:19,2015-05-01 12:57:19
4,593847687847350272,Watching a thread on FB about possible future ...,"<a href=""http://tapbots.com/tweetbot"" rel=""nof...",6.780330e+05,NaN,0.0,0.0,NaN,0.000000e+00,NaN,...,NaN,NaN,NaN,0.0,0.0,0.0,Thu Apr 30 18:41:36 +0000 2015,2015-04-30 20:41:36,2015-05-01 12:57:19,2015-05-01 12:57:19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2839356,546418822677532673,"""When a woman no longer gets frustrated and up...","<a href=""http://www.tumblr.com/"" rel=""nofollow...",2.776254e+07,NaN,0.0,0.0,NaN,0.000000e+00,NaN,...,NaN,NaN,NaN,0.0,1.0,0.0,Sat Dec 20 21:36:13 +0000 2014,2014-12-20 22:36:13,2015-05-01 16:15:29,2015-05-01 16:15:29
2839357,546379158251929600,"""Hereâs the key to living a good life: âMe...","<a href=""http://www.tumblr.com/"" rel=""nofollow...",2.776254e+07,NaN,0.0,0.0,NaN,0.000000e+00,NaN,...,NaN,NaN,NaN,0.0,1.0,0.0,Sat Dec 20 18:58:37 +0000 2014,2014-12-20 19:58:37,2015-05-01 16:15:29,2015-05-01 16:15:29
2839358,578751592213700608,RT @Cuauhtemoc_1521: #SabiasQue el nombre real...,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",2.356426e+09,NaN,0.0,0.0,NaN,5.786470e+17,NaN,...,NaN,NaN,NaN,1.0,0.0,1.0,Fri Mar 20 02:55:06 +0000 2015,2015-03-20 03:55:06,2015-05-01 16:15:29,2015-05-01 16:15:29
2839359,546379003465314306,"""If someone says they have their shit together...","<a href=""http://www.tumblr.com/"" rel=""nofollow...",2.776254e+07,NaN,0.0,0.0,NaN,0.000000e+00,NaN,...,NaN,NaN,NaN,0.0,1.0,0.0,Sat Dec 20 18:58:00 +0000 2014,2014-12-20 19:58:00,2015-05-01 16:15:29,2015-05-01 16:15:29


In [ ]:
retweets = gen_tweets[gen_tweets['retweeted_status_id']!=0][['text']]

In [ ]:
retweets.reset_index(drop = True, inplace = True)

In [ ]:
retweets.shape

(738089, 1)

In [ ]:
retweets

,text
0,RT @morningJewshow: Speaking about Jews and co...
1,"RT @SanFranciscoBA: #enoughsenough ""Surprise! ..."
2,"RT @thepatrickwalsh: Kriss Kross once rapped ""..."
3,RT @TheAtlanticPOL: Watch Baltimore native @ta...
4,RT @sippey: Here's my one paragraph review of ...
...,...
738084,RT @NotableHistory: A British Bren Gun Carrier...
738085,RT @_Buddha_Quotes: Free yourself from pleasur...
738086,RT @phatn3s1an: I hope everyone is safe and we...
738087,RT @_Buddha_Quotes: You too shall pass away. K...


In [ ]:
retweets[retweets['text'].str.contains("RT")==False]

,text
317527,This Tweet from @TheAnkitDalal has been withhe...
569984,This Tweet from @A1_Ade has been withheld in r...
626014,This Tweet from @Dilly_Wonka has been withheld...
641322,This Tweet from @laurrkat has been withheld in...


In [ ]:
print(retweets.loc[317527]['text'])

This Tweet from @TheAnkitDalal has been withheld in response to a report from the copyright holder.


In [ ]:
replies = gen_tweets[gen_tweets['in_reply_to_status_id'] != 0][['text']]

In [ ]:
replies.reset_index(drop = True, inplace = True)

In [ ]:
replies

,text
0,@thelancearthur @hotdogsladies I didn't realiz...
1,@wkamaubell I stand by you. I don't want men t...
2,@kevinroose This got funded during this first ...
3,@studiomondeo are you sticking with @mailbox o...
4,@EllenPapadakis on a completely separate note
...,...
752290,@Dj_Podyum @TES_acoelojo sedentaria y grasas t...
752291,@Dj_Podyum @TES_acoelojo Â¿Y quÃ© serÃ­a de lo...
752292,@TO @Lean Email on its way...
752293,"@AliLeonardMC @brekkydude you got the ""little""..."


In [ ]:
replies[replies['text'].str.contains('@') == False]

,text
862,12/ Notice in the AP tape that the most powerf...
863,11/ This is so important to know. Wonderful pe...
864,"10/ Then see if any of them ever told friends,..."
865,10/ People (you can see it on twitter already)...
870,"9/ notice Camille his wife how she's there, ho..."
...,...
752068,"Ayer parecÃ­an las clases, clases optativas"
752069,#porahora
752070,* y no querÃ­an petarme el culo y/o robarme la...
752075,temazÃ³n!!!!!!


In [ ]:
gen_tweets[(gen_tweets['in_reply_to_status_id'] == 0) & (gen_tweets['retweeted_status_id']==0)]['text']

1          This age/face recognition thing..no reason pla...
2          Only upside of the moment I can think of is th...
3          If you're going to think about+create experien...
4          Watching a thread on FB about possible future ...
5                         Don't. Ok? https://t.co/uTXrJ6zvdN
                                 ...                        
2839355    Is it true @brekkydude?  2014 is going to be h...
2839356    "When a woman no longer gets frustrated and up...
2839357    "Hereâs the key to living a good life: âMe...
2839359    "If someone says they have their shit together...
2839360    How to Train Your Coral: Imagine you are in th...
Name: text, Length: 1348562, dtype: object